# AFL End of Year Process
After the year has eneded, we can reset all the data ready for the new year and consider the parameters of our model.
<br>Will have to update some season, year and dealing with whatever fun season change the powers that be dream up.<br>This is the playground essentially.

In [1]:
#import packages
import polars as pl
import numpy as np
#Import replace Team Names form other file so I only have to update in one place
from ipynb.fs.defs.AFLupdate import replaceTeamNames
pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_rows(1000);
pl.Config.set_tbl_cols(40);

/home/erin/Documents/AFL tipping/afl-env/lib/python3.12/site-packages/rpy2/robjects/lib/dplyr.py:27: UserWarning: This was designed against dplyr versions starting with 1.0 but you have 1.1.4
  warnings.warn(


In [2]:
#set up R extension
%load_ext rpy2.ipython

Get the fixtures from 2015>, taking into account the bye infomation 2019 onwards. 

In [3]:
%%R -o seasons,seasonBye
library('fitzRoy')
library('dplyr')
library('tidyr')

seasonList <- c(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025)
first <- TRUE
for(s in seasonList) {
    
    df <- fetch_fixture(season = s, comp = "AFLM")

    if (first) {
        seasons <- df
        first <- FALSE
    }else{
        seasons <- seasons %>% 
                      bind_rows(df)
    }
    
}

seasonBye <- seasons %>% 
                select(compSeason.id, compSeason.year, round.roundNumber, round.byes) %>%
                filter(compSeason.year >= 2019) %>%
                unnest(round.byes) %>%
                distinct() %>%
                mutate(round.nextRound = round.roundNumber + 1,
                      ByeFlag = 1) %>%
                select(compSeason.id, round.nextRound, club.id, club.name, ByeFlag)

seasons <- seasons %>% select(-round.byes)
    

ℹ Returning data for "All Rounds, 2015"
✔ Returning data for "All Rounds, 2015" s]

ℹ Returning data for "All Rounds, 2016"
✔ Returning data for "All Rounds, 2016" s]

ℹ Returning data for "All Rounds, 2017"
✔ Returning data for "All Rounds, 2017" s]

ℹ Returning data for "All Rounds, 2018"
✔ Returning data for "All Rounds, 2018" s]

ℹ Returning data for "All Rounds, 2019"
✔ Returning data for "All Rounds, 2019" s]

ℹ Returning data for "All Rounds, 2020"
✔ Returning data for "All Rounds, 2020" s]

ℹ Returning data for "All Rounds, 2021"
✔ Returning data for "All Rounds, 2021" s]

ℹ Returning data for "All Rounds, 2022"
✔ Returning data for "All Rounds, 2022" s]

ℹ Returning data for "All Rounds, 2023"
✔ Returning data for "All Rounds, 2023" s]

ℹ Returning data for "All Rounds, 2024"
✔ Returning data for "All Rounds, 2024" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]



In addition: Warning messages:
1: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages
2: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages
3: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages
4: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages


Assign -1 as the winner id for a draw, since 2015 there have been 15 draws. We also have a cancelled game, Cats vs. Crows 2015 R14 which we are just going to remove.

In [4]:
#Clean and prep the season bye info
seasonBye = pl.from_pandas(seasonBye).drop('club.id').with_columns(pl.col('round.nextRound').cast(pl.Int32))

In [5]:
#Ensure all the club names are correct
seasonBye = replaceTeamNames(seasonBye, 'club.name')

In [6]:
#get the correct team names for the seasons
seasons = pl.from_pandas(seasons).with_columns(pl.col('home.team.name').str.strip_chars())
seasons = replaceTeamNames(seasons, 'home.team.name')
seasons = replaceTeamNames(seasons, 'away.team.name')

In [7]:
#The inital cleaning working with the gneral match details. We can use this as our base for creating the desired attributes
seasonsBase = (seasons
               #remove 2025 (current round that have occured)
                .filter((pl.col('compSeason.year') != 2025) | (pl.col('round.roundNumber') <= 9))
                .sort('round.id')
                #Keep only the columns we want
                .select('id', 'status', 'compSeason.id', 'compSeason.year', 'round.roundNumber', 'home.team.club.id', 'home.team.name',
                        'away.team.club.id', 'away.team.name', 'home.score.goals', 'home.score.behinds', 'home.score.totalScore',
                        'away.score.goals', 'away.score.behinds', 'away.score.totalScore', 'venue.providerId', 'venue.name', 'venue.state')
                #Remove the Crows vs. Cats 2015 R14 game which was cancelled
                .filter(pl.col('id') != 847)
                #Label Previous comp year
                .with_columns((pl.col('compSeason.year') - 1).alias('prevComp.year'))
                #rename venueId
                .rename({'venue.providerId':'venue.id'})
                #Label in sequential order (currently missing numbers in existing round.id) to get the previous games 
                .join(seasons
                .select('compSeason.year', 'round.roundNumber')
                .unique()
                .sort('compSeason.year', 'round.roundNumber')
                .with_columns(pl.col('round.roundNumber').cum_count().alias('round.id')), how='left', on=['compSeason.year', 'round.roundNumber'])
     #create the margin
        .with_columns((pl.col('home.score.totalScore') - pl.col('away.score.totalScore')).abs().alias('win.margin'))   
     #Add a bye flag
        .join(seasonBye, how='left', left_on=['compSeason.id', 'round.roundNumber', 'home.team.name'], right_on=['compSeason.id', 'round.nextRound', 'club.name'])
        .join(seasonBye, how='left', left_on=['compSeason.id', 'round.roundNumber', 'away.team.name'], right_on=['compSeason.id', 'round.nextRound', 'club.name'])
     #Rename bye flag
        .rename({'ByeFlag' : 'home.bye.flag',
              'ByeFlag_right' : 'away.bye.flag'})
     #Fill in bye flag nulls
        .with_columns(pl.col('home.bye.flag').fill_null(0).cast(pl.Int32),
                   pl.col('away.bye.flag').fill_null(0).cast(pl.Int32))
    #Fill venue blank states with china (1 is not china but NZ but is in 2015 so we wont worry about that)
        .with_columns(pl.col('venue.state').fill_null('China'))
        )

In [8]:
#Add a round id
seasonsBase = (seasonsBase
     .join((seasonsBase
                 .select('compSeason.year', 'round.roundNumber')
                 .unique()
                 .sort('compSeason.year', 'round.roundNumber')
                 .with_columns(pl.col('round.roundNumber').cum_count().alias('round.Id'))), how='left', on=['compSeason.year', 'round.roundNumber'])
)

In [9]:
#Create a dataframe with all the match details (repeated so that boths teams get to be team A and team B)
allMatchesBothSides = (seasonsBase
     .select('compSeason.year', 'round.roundNumber', 'round.id', 'venue.id', 'home.team.club.id', 'home.team.name', 'home.score.totalScore', 
             'home.score.goals', 'home.score.behinds', 'away.team.club.id', 'away.team.name', 'away.score.totalScore', 'away.score.goals', 'away.score.behinds')
     .rename({'home.team.name': 'team.A.name',
              'home.team.club.id' : 'team.A.club.id',
              'home.score.totalScore' : 'team.A.score.totalScore',
              'home.score.goals' : 'team.A.score.goals',
              'home.score.behinds' : 'team.A.score.behinds',
              'away.team.name': 'team.B.name',
              'away.team.club.id' : 'team.B.club.id',
              'away.score.totalScore' : 'team.B.score.totalScore',
              'away.score.goals' : 'team.B.score.goals',
              'away.score.behinds' : 'team.B.score.behinds'})
     .vstack(seasonsBase
     .select('compSeason.year', 'round.roundNumber', 'round.id', 'venue.id', 'away.team.club.id', 'away.team.name', 'away.score.totalScore',
             'away.score.goals', 'away.score.behinds', 'home.team.club.id', 'home.team.name', 'home.score.totalScore', 'home.score.goals', 'home.score.behinds')
     .rename({'home.team.name': 'team.B.name',
              'home.team.club.id' : 'team.B.club.id',
              'home.score.totalScore' : 'team.B.score.totalScore',
              'home.score.goals' : 'team.B.score.goals',
              'home.score.behinds' : 'team.B.score.behinds',
              'away.team.name': 'team.A.name',
             'away.team.club.id' : 'team.A.club.id',
              'away.score.totalScore' : 'team.A.score.totalScore',
              'away.score.goals' : 'team.A.score.goals',
              'away.score.behinds' : 'team.A.score.behinds'}))
     .sort('team.A.club.id', 'round.id')
)

In [10]:
#Create a link between teams club id and the elo index 0-17 
indexTeams = allMatchesBothSides.select('team.A.name', 'team.A.club.id').unique().sort('team.A.club.id').with_columns((pl.col('team.A.name').cum_count() - 1).alias('EloIndex'))

In [11]:
#Add the created team index
seasonsBase = (seasonsBase
                   .join(indexTeams.drop('team.A.club.id'), how='left', left_on='home.team.name', right_on='team.A.name')
                   .rename({'EloIndex' : 'home.team.id'})
                   .join(indexTeams.drop('team.A.club.id'), how='left', left_on='away.team.name', right_on='team.A.name')
                   .rename({'EloIndex' : 'away.team.id'})
                    #Label the id winner (-1 for a draw, 1 for home & 0 for away)
                    .with_columns(pl.when(pl.col('home.score.totalScore') >= pl.col('away.score.totalScore')).then(pl.lit(1))
                                .when(pl.col('home.score.totalScore') < pl.col('away.score.totalScore')).then(pl.lit(0))
                                 .alias('winner'))
              )

## Get the ladder poistions at beginning round and last seasons final ladder position.

In [12]:
%%R -o ladder

season <- c(2015,2016,2017,2018,2019,2020,2021, 2022, 2023, 2024, 2025)
rounds <- c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23)
first <- TRUE

for(s in season){
    for(r in rounds){

        if(s != 2025 | r <= 9){

            if(s==2020 && r <= 18){
                df <- fetch_ladder_afl(season=s, round_number=r)
            }else if(s!=2020){
                df <- fetch_ladder_afl(season=s, round_number=r)
            }

            if(first){
                ladder <- df
                first <- FALSE
            }else{
                ladder <- ladder %>% bind_rows(df)
            }
        }
    }

    if(s == 2024 | s == 2025){
        ladder <- ladder %>% bind_rows(fetch_ladder_afl(season=s, round_number = 0))
    }

    if(s >= 2023){
        ladder <- ladder %>% bind_rows(fetch_ladder_afl(season=s, round_number=24))
    }
}

ladder = ladder %>% select(season, round_number, position, team.club.id, team.club.name)
        

In addition: Warning message:
No data found for specified round number and season 


In [13]:
#Ensure ladder team names are correct
ladder = replaceTeamNames(pl.from_pandas(ladder), 'team.club.name')

In [14]:
ladderNormal = (ladder
     .with_columns((pl.col('round_number') + 1).alias('nextRound'))
     .drop('team.club.id', 'round_number')
                  )

In [15]:
#get the final ladder of each season
ladderFinal = (ladder
                   #Get the final ladder at the end of the season
                   .filter(pl.col('round_number') == pl.col('round_number').max().over('season'))
                   #get the next season (for assigning to the nest round 1)
                   .with_columns((pl.col('season') + 1).alias('Nextseason'),
                                 pl.lit(1).alias('first_round'))
              )

In [16]:
#Add home and away ladder positions before 
ladderPosition = (seasonsBase
     .select('compSeason.year', 'round.roundNumber')
     .unique()
     .sort('compSeason.year', 'round.roundNumber')
     #from round 2 till finals
     .join(ladderNormal, how='left', left_on=['compSeason.year', 'round.roundNumber'], right_on=['season', 'nextRound'])
     #for round 1 the final ladder of previous season
     .join(ladderFinal.drop('season', 'round_number'), how='left', left_on=['compSeason.year', 'round.roundNumber'], right_on=['Nextseason', 'first_round'])
     .rename({'position_right' : 'position1',
              'team.club.name_right' : 'team.club.name1'})
     #for the finals
     .join(ladderFinal, how='left', left_on='compSeason.year', right_on='season')
     #put all the parts together
     .with_columns(pl.when(~pl.col('position').is_null()).then(pl.col('position'))
                     .when(~pl.col('position1').is_null()).then(pl.col('position1')).otherwise(pl.col('position_right')).alias('position'),
                   pl.when(~pl.col('team.club.name').is_null()).then(pl.col('team.club.name'))
                     .when(~pl.col('team.club.name1').is_null()).then(pl.col('team.club.name1')).otherwise(pl.col('team.club.name_right')).alias('team.club.name'))
     #select only what we want
     .select('compSeason.year', 'round.roundNumber', 'team.club.name', 'position')
     .unique()
     .sort('compSeason.year', 'round.roundNumber', 'position')
     #get the fianl positioning of the previous year
     .join(ladderFinal.drop('season', 'round_number', 'first_round', 'team.club.id'), how='left', left_on=['compSeason.year', 'team.club.name'], right_on=['Nextseason', 'team.club.name'])
     .rename({'position_right': 'last.season.position'})
                           
)           

In [17]:
seasonsStep2 = (seasonsBase
                 #join the ladderPosition to get the home teams ladder info
                 .join(ladderPosition.rename({'position':'home.ladder.position', 'last.season.position': 'home.last.season.position'}),
                       how='left',
                       left_on=['compSeason.year', 'round.roundNumber', 'home.team.name'],
                       right_on=['compSeason.year', 'round.roundNumber', 'team.club.name'])
                 #Clean up any duplicates
                 .unique()
                 #join the ladderPosition to get the away teams ladder info
                 .join(ladderPosition.rename({'position':'away.ladder.position', 'last.season.position': 'away.last.season.position'}),
                       how='left',
                       left_on=['compSeason.year', 'round.roundNumber', 'away.team.name'],
                       right_on=['compSeason.year', 'round.roundNumber', 'team.club.name'])
                 #clean up any duplicates
                 .unique()
                 #sort into order
                 .sort('compSeason.year', 'round.roundNumber')
     
                )

## Get the distance travelled

In [18]:
#Create a table for distance between states and where the team is based (based on capital to capital)
vic = pl.Series([0, 654, 714, 2727, 1373, 0, 0, 1373, 0, 0, 0, 654, 0, 0, 0, 2727, 714, 0])
nsw = pl.Series([714, 1165, 0, 3297, 730, 714, 714, 730, 714, 714, 714, 1165, 714, 714, 714, 3297, 0, 714])
qld = pl.Series([1373, 1602, 730, 3613, 0, 1373, 1373, 0, 1373, 1373, 1373, 1602, 1373, 1373, 1373, 3613, 730, 1373])
nt = pl.Series([3140, 2609, 3144, 2647, 2846, 3140, 3140, 2846, 3140, 3140, 3140, 2609, 3140, 3140, 3140, 2647, 3144, 3140])
wa = pl.Series([2727, 2135, 3297, 0, 3613, 2727, 2727, 3613, 2727,2727, 2727, 2135, 2727, 2727, 2727, 0, 3297, 2727])
sa = pl.Series([654, 0, 1165, 2135, 1602, 654, 654, 1602, 654, 654, 654, 0, 654, 654, 654, 2135, 1165, 654])
tas = pl.Series([597, 1161, 1056, 3015, 1786, 597, 597, 1786, 597, 597, 597, 1161, 597, 597, 597, 3015, 1161, 597])
act = pl.Series([467, 960, 942, 3095, 942, 467, 467, 247, 467, 467, 467, 960, 467, 467, 467, 3095, 247, 467])
china = pl.Series([8036, 6956, 7592, 7050, 6225, 8036, 8036, 6225, 8036, 8036, 8036, 6956, 8036, 8036, 8036, 7050, 7592, 8036])
distanceTravelled = (seasonsBase
                         .select('home.team.name', 'home.team.club.id').unique()
                         .sort('home.team.club.id')
                         .with_columns(vic.alias('VIC'),
                                       nsw.alias('NSW'),
                                       qld.alias('QLD'),
                                       nt.alias('NT'),
                                       wa.alias('WA'),
                                       sa.alias('SA'),
                                       tas.alias('TAS'),
                                       act.alias('ACT'),
                                       china.alias('China'))
                         .rename({'home.team.name' : 'team.name'})
                         .drop('home.team.club.id')
                         .melt(id_vars='team.name', variable_name='State', value_name='Distance')
                        )

/tmp/ipykernel_75370/2175747362.py:25: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  .melt(id_vars='team.name', variable_name='State', value_name='Distance')


In [19]:
seasonStep3 = (seasonsStep2
                 #Home team distance travelled to venue
                 .join(distanceTravelled.rename({'Distance' : 'home.distance'}), how='left', left_on=['home.team.name', 'venue.state'], right_on=['team.name', 'State'])
                 #Away team distance travelled to venue
                 .join(distanceTravelled.rename({'Distance' : 'away.distance'}), how='left', left_on=['away.team.name', 'venue.state'], right_on=['team.name', 'State'])
                )

In [20]:
seasonStep3

id,status,compSeason.id,compSeason.year,round.roundNumber,home.team.club.id,home.team.name,away.team.club.id,away.team.name,home.score.goals,home.score.behinds,home.score.totalScore,away.score.goals,away.score.behinds,away.score.totalScore,venue.id,venue.name,venue.state,prevComp.year,round.id,win.margin,home.bye.flag,away.bye.flag,round.Id,home.team.id,away.team.id,winner,home.ladder.position,home.last.season.position,away.ladder.position,away.last.season.position,home.distance,away.distance
i32,str,i32,f64,i32,i32,str,i32,str,i32,i32,i32,i32,i32,i32,str,str,str,f64,u32,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i64,i64
732,"""CONCLUDED""",7,2015.0,1,14,"""Melbourne""",9,"""Gold Coast Suns""",17,13,115,13,11,89,"""CD_V40""","""MCG""","""VIC""",2014.0,1,26,0,0,1,8,4,1,13,null,16,null,0,1373
741,"""CONCLUDED""",7,2015.0,1,13,"""Brisbane Lions""",12,"""Collingwood""",11,8,74,12,14,86,"""CD_V20""","""Gabba""","""QLD""",2014.0,1,12,0,0,1,7,6,0,17,null,12,null,0,1373
726,"""CONCLUDED""",7,2015.0,1,25,"""Western Bulldogs""",23,"""West Coast Eagles""",14,13,97,14,3,87,"""CD_V190""","""Marvel Stadium""","""VIC""",2014.0,1,10,0,0,1,17,15,1,6,null,2,null,0,2727
723,"""CONCLUDED""",7,2015.0,1,24,"""Sydney Swans""",10,"""Essendon""",10,12,72,9,6,60,"""CD_V9""","""Accor Stadium""","""NSW""",2014.0,1,12,0,0,1,16,5,1,4,null,15,null,0,714
731,"""CONCLUDED""",7,2015.0,1,6,"""Fremantle""",19,"""Port Adelaide""",11,9,75,10,8,68,"""CD_V70""","""Domain Stadium""","""WA""",2014.0,1,7,0,0,1,3,11,1,1,null,9,null,0,2135
727,"""CONCLUDED""",7,2015.0,1,1,"""Carlton""",22,"""Richmond""",11,12,78,15,15,105,"""CD_V40""","""MCG""","""VIC""",2014.0,1,27,0,0,1,0,14,0,18,null,5,null,0,0
725,"""CONCLUDED""",7,2015.0,1,21,"""St Kilda""",5,"""GWS Giants""",11,12,78,12,15,87,"""CD_V190""","""Marvel Stadium""","""VIC""",2014.0,1,9,0,0,1,13,2,0,14,null,11,null,0,714
742,"""CONCLUDED""",7,2015.0,1,3,"""Adelaide Crows""",20,"""North Melbourne""",21,14,140,9,9,63,"""CD_V6""","""Adelaide Oval""","""SA""",2014.0,1,77,0,0,1,1,12,1,7,null,8,null,0,654
729,"""CONCLUDED""",7,2015.0,1,16,"""Hawthorn""",15,"""Geelong Cats""",17,21,123,8,13,61,"""CD_V40""","""MCG""","""VIC""",2014.0,1,62,0,0,1,10,9,1,3,null,10,null,0,0


## Team ELO Rating

In [21]:
#create a function to find a teams prediction
def eloTeamPrediction(ratingTeamA, ratingTeamB):
    exp = (-1*(ratingTeamA - ratingTeamB))/400
    b = 1 + 10 ** exp
    result = 1/b
    return result

In [22]:
#create a function to find the teams result, inital B = 0.004 per Multifactorial analysis of factors influencing elite australian football match outcomes: a machine learning approach
def eloTeamResult(scoreTeamA, scoreTeamB):
    exp = -0.04*(scoreTeamA - scoreTeamB)
    b = 1 + np.exp(exp)
    result = 1/b
    return result

In [23]:
#Create a function to get change in teams elo, set K=67.559 per Multifactorial analysis of factors influencing elite australian football match outcomes: a machine learning approach
def eloChange(ratingTeamA, ratingTeamB, scoreTeamA, scoreTeamB):
    #get Team prediction
    prediction = eloTeamPrediction(ratingTeamA, ratingTeamB)
    
    #get actual result
    actual = eloTeamResult(scoreTeamA , scoreTeamB)
    
    #get elo change
    result = 20*(actual - prediction)
    
    #get new elo
    newElo = ratingTeamA + result
    
    return newElo

In [24]:
#Create a function to get the elo for the start of the season, set Carry over at 0.70628 per Multifactorial analysis of factors influencing elite australian football match outcomes: a machine learning approach
def eloNewSeason(lastElo):
    carryOver = 0.80
    result = carryOver * lastElo + 1500 * (1 - carryOver)
    return result

In [25]:
#Function to rule them all! assign elo values
def eloFucntions(roundNumber, teamAScore, teamBScore, prevEloA, prevEloB):
    #If first round of the season get the carry over elo value
    if roundNumber == 1:
        elo = eloNewSeason(prevEloA)
    #If normal round then find new score
    else:
        elo = eloChange(prevEloA, prevEloB, teamAScore, teamBScore)
    #return current elo 
    return elo   
   
    

In [26]:
#Elo list of lists to record elo's and access past elo values, with index = elo index translation found in eloIndexTeams table
#Starting elo as default 1500
eloScores = [[1500] * 18]
#starting roundIndex 

In [27]:
def eloNumber(roundId, roundNumber, clubA, clubB, roundIdA, roundIdB, indexA, indexB):
    #set roundId for indexing
    roundId = roundId - 1
    #create default elo
    elo=1500.0
    #Check if new round by checking a new list has been added for the round
    if roundId >= np.shape(eloScores)[0]:
        #Append new list for new round
        eloScores.append(eloScores[roundId - 1])
        
    #Deal with very first round we consider, simply create base for next round
    if roundId == 0:
        elo = 1500.0
    #For all other rounds get the elo
    else: 
        
        #Get the new elo score
        elo = eloFucntions(roundNumber, teamScores[clubA][roundIdA-1], teamScores[clubB][roundIdB-1], eloScores[roundId-1][indexA], eloScores[roundId-1][indexB])
        #Change list of list to reflect the new score
        eloScores[roundId][indexA] = elo
        

    #Return Elo
    return elo
        

In [28]:
teamScores = dict(allMatchesBothSides.drop('team.B.club.id', 'team.B.name', 'team.B.score.totalScore').group_by('team.A.name').agg(pl.col('team.A.score.totalScore')).to_numpy())

In [29]:
eloScoreDF = (allMatchesBothSides
                 #Get the cleaned indexes for A teams
                 .join(indexTeams.rename({'EloIndex': 'elo.index.A'}).drop('team.A.club.id'), how='left', on='team.A.name')
                 #Get the cleaned indexes for B teams
                 .join(indexTeams.rename({'EloIndex': 'elo.index.B'}).drop('team.A.club.id'), how='left', left_on='team.B.name', right_on='team.A.name')
                 .with_columns((pl.col('team.A.name').cum_count() - 1).over(pl.col('team.A.name')).alias('team.A.round.index'))
                 .sort('round.id', 'team.B.name')
                 .with_columns((pl.col('team.B.name').cum_count() - 1).over(pl.col('team.B.name')).alias('team.B.round.index'))
                 #sort so the assigning of elo scores are pasted on past club info
                 .sort('round.id', 'team.A.name')
                 #Get the elo scores
                 .with_columns(pl.struct(['round.roundNumber', 'round.id', 'team.A.name', 'team.B.name', 'team.A.round.index', 'team.B.round.index', 'elo.index.A', 'elo.index.B'])
                                 .map_elements(lambda x : eloNumber(x['round.id'], x['round.roundNumber'], x['team.A.name'], x['team.B.name'], x['team.A.round.index'], x['team.B.round.index'], x['elo.index.A'], x['elo.index.B']))
                                 .alias('elo.score'))
                 #Keep only the info we actually want
                 .select('compSeason.year', 'round.roundNumber', 'round.id', 'team.A.name', 'elo.score')
                )

/tmp/ipykernel_75370/1401470024.py:12: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(pl.struct(['round.roundNumber', 'round.id', 'team.A.name', 'team.B.name', 'team.A.round.index', 'team.B.round.index', 'elo.index.A', 'elo.index.B'])


In [30]:
#Join the elo scores up into the main dataframe
seasonStep3 = (seasonsStep2
     #elo scores for the home teams
     .join(eloScoreDF.rename({'elo.score' : 'home.elo.score'}),
           how='left',
           left_on=['compSeason.year', 'round.roundNumber', 'round.id', 'home.team.name'],
           right_on=['compSeason.year', 'round.roundNumber', 'round.id', 'team.A.name'])
     #elo scores for the away teams
    .join(eloScoreDF.rename({'elo.score' : 'away.elo.score'}),
           how='left',
           left_on=['compSeason.year', 'round.roundNumber', 'round.id', 'away.team.name'],
           right_on=['compSeason.year', 'round.roundNumber', 'round.id', 'team.A.name'])
)

## Home Advantage/Disadvantage
Turns out (even not considering 2020, which we are) the term 'home team' can mean fairly little which might go some way to explaining how the chances of winning if the home team is only slightly elvated due to been the home team. Therefore to consider this, we will get the portion of games each time played at the venue in the previous year, this of course can mean that there are some venues that no-one played at last year and the advantage/disadvantage will be 0 (and not undefinied).<br>
2025: Update to percent of wins at the ground, grouping by rolling seasons, i.e. 26 rounds instead of striaght season.

In [31]:
#Get the portion of games played at venue current season
teamRoundVenue = (allMatchesBothSides
                     #index the team names for rolling 
                     .sort('team.A.name', 'compSeason.year', 'round.id')
                     .with_columns(pl.col('team.A.name').cum_count().over('team.A.name').alias('team.A.index'))
                     #.sort('team.A.index', 'team.A.name')               
                     #find the winner of each game 
                     .with_columns(pl.when(pl.col('team.A.score.totalScore') >= pl.col('team.B.score.totalScore')).then(pl.lit(1)).otherwise(pl.lit(0)).alias('team.A.winner'))
                     #Get the number of times each team played at a venue during past 26 rounds approx. number of games a season i.e. rolling season
                     .sort('team.A.name', 'round.id'))

teamRoundVenue = (teamRoundVenue
                     .rolling(index_column='team.A.index', period='26i', closed='left', group_by=['team.A.name', 'venue.id'])
                     .agg(pl.col('round.id').len().alias('VenuePlayed'),
                          #pl.col('compSeason.year').max().alias('compSeason.year'),
                          #pl.col('round.id').max().alias('round.id'),
                          pl.col('team.A.winner').sum().alias('wins'))
                     .join(teamRoundVenue.select('compSeason.year', 'round.id', 'team.A.name', 'team.A.index'), how='left', on=['team.A.name', 'team.A.index'])
                     .unique()
                     #How much of their time did the team spend at any venue
                     .with_columns((pl.col('VenuePlayed')/26).alias('VenuePortion'))
                     #.with_columns((pl.col('wins')/pl.col('VenuePlayed')).alias('VenueWinnerPortion'))
                     #.with_columns((pl.col('VenuePortion')*pl.col('VenueWinnerPortion')).alias('VenuePortion'))
                     .sort('compSeason.year', 'team.A.name', 'VenuePlayed')
                     #Keep only what we need going forward
                     .select('round.id', 'team.A.name', 'venue.id', 'VenuePortion')#, 'VenueWinnerPortion')
                     .with_columns(pl.col('VenuePortion').fill_null(0))
                     .with_columns(pl.col('VenuePortion').fill_nan(0))        
                 )

In [32]:
#Attach back to the seasonsStep3 and calculate the home advatage and disadvantage
seasonStep4 = (seasonStep3
     #join portion for home teams
     .join(teamRoundVenue, how='left', left_on=['round.id', 'home.team.name', 'venue.id'], right_on=['round.id', 'team.A.name', 'venue.id'])
     #join portion for away.teams
     .join(teamRoundVenue, how='left', left_on=['round.id', 'away.team.name', 'venue.id'], right_on=['round.id', 'team.A.name', 'venue.id'])
     #claify the column names
     .rename({'VenuePortion' : 'home.venue.portion',
              #'VenueWinnerPortion': 'home.venue.winner',
              'VenuePortion_right' : 'away.venue.portion'})
              #'VenueWinnerPortion_right': 'away.venue.winner'})
)

## Days Between Matches
Have to go in and get afttables match info to get dates (would this work with upcoming matches, may need to be revised if not)

In [33]:
%%R -o seasons
seasonList <- c(2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025)
#seasonList <- c(2015)
first <- TRUE
for(s in seasonList) {
    
    df <- fetch_fixture(season = s, comp = "AFLM")

    if (first) {
        seasons <- df
        first <- FALSE
    }else{
        seasons <- seasons %>% 
                      bind_rows(df)
    }
}

seasons <- seasons  %>%
    select('utcStartTime', 'compSeason.year', 'round.roundNumber', 'home.team.name', 'away.team.name')

ℹ Returning data for "All Rounds, 2014"


✔ Returning data for "All Rounds, 2014" s]

ℹ Returning data for "All Rounds, 2015"
✔ Returning data for "All Rounds, 2015" s]

ℹ Returning data for "All Rounds, 2016"
✔ Returning data for "All Rounds, 2016" s]

ℹ Returning data for "All Rounds, 2017"
✔ Returning data for "All Rounds, 2017" s]

ℹ Returning data for "All Rounds, 2018"
✔ Returning data for "All Rounds, 2018" s]

ℹ Returning data for "All Rounds, 2019"
✔ Returning data for "All Rounds, 2019" s]

ℹ Returning data for "All Rounds, 2020"
✔ Returning data for "All Rounds, 2020" s]

ℹ Returning data for "All Rounds, 2021"
✔ Returning data for "All Rounds, 2021" s]

ℹ Returning data for "All Rounds, 2022"
✔ Returning data for "All Rounds, 2022" s]

ℹ Returning data for "All Rounds, 2023"
✔ Returning data for "All Rounds, 2023" s]

ℹ Returning data for "All Rounds, 2024"
✔ Returning data for "All Rounds, 2024" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]



In [34]:
#Ensure that seasons have correct team names
seasons = pl.from_pandas(seasons)
seasons = replaceTeamNames(seasons, 'home.team.name')
seasons = replaceTeamNames(seasons, 'away.team.name')


In [35]:
#Get the dates into python and convert the datetime into just a date
matchDate = (seasons
                 .with_columns(pl.col('utcStartTime').str.to_datetime().dt.date().alias('Date'))
                 .unpivot(index=['Date', 'compSeason.year', 'round.roundNumber'], on=['home.team.name', 'away.team.name'], value_name='Team', variable_name='Remove')
                 .drop('Remove')
                 .sort('Team', 'Date')
                 .with_columns(pl.col('Team').cum_count().over('Team').alias('team.round.id'))
            )

In [36]:
#Create the date of the past date for each team
matchDate2 = (matchDate
                 #Get the last match id, setting the very first to indicate that there was no match previous
                 .with_columns(pl.when(pl.col('team.round.id') == 1).then(pl.lit(0)).otherwise(pl.col('team.round.id') - 1).alias('team.round.id.past'))
                 #join with itself to get the last match date
                 .join(matchDate.drop('compSeason.year'), how='left', left_on=['Team', 'team.round.id.past'], right_on=['Team', 'team.round.id'])
                 .rename({'Date_right' : 'date.last'})
                 #find days difference between the two games
                 .with_columns((pl.col('Date') - pl.col('date.last')).dt.total_days().alias('day.since.match'))
                 #keep only what I want
                 .select('compSeason.year', 'round.roundNumber', 'Team', 'day.since.match')        
            )

In [37]:
#Add the days back to main table
#
seasonStep5 = (seasonStep4
                 #join the home team days since last played
                 .join(matchDate2, how='left', left_on=['compSeason.year', 'round.roundNumber', 'home.team.name'], right_on=['compSeason.year', 'round.roundNumber', 'Team'])
                 .rename({'day.since.match' : 'home.day.last.match'})
                 #join the awway team days since last played
                 .join(matchDate2, how='left', left_on=['compSeason.year', 'round.roundNumber', 'away.team.name'], right_on=['compSeason.year', 'round.roundNumber', 'Team'])
                 .rename({'day.since.match' : 'away.day.last.match'})
                )

## Attacking, defending and opposition Strength
AFL has both goals and behinds so, we will count goals and add give points a value of 0.5 to reflect that they are of lesser value to the game than a goal.

In [38]:
#Get the goals scored and given away and the margin of each game. Add in the ladder positions as well
allMatchesBothSidesStats = (allMatchesBothSides
                 #Goals scored
                 .with_columns((pl.col('team.A.score.goals') + (pl.col('team.A.score.behinds')*0.5)).alias('strength.attack'))
                 #Goals given away
                 .with_columns((pl.col('team.B.score.goals') + (pl.col('team.B.score.behinds')*0.5)).alias('strength.defense'))
                 #goal diff , i.e. margin
                 #.with_columns((pl.col('team.A.score.totalScore') - pl.col('team.B.score.totalScore')).abs().alias('margin'))
                 #keep only what I want
                 .select('compSeason.year', 'round.roundNumber', 'round.id', 'team.A.club.id', 'team.A.name', 'team.B.club.id', 'team.B.name',
                         'strength.attack', 'strength.defense', 'team.A.score.totalScore', 'team.B.score.totalScore')
                 #Cast round.id to int rather an unsigned int
                 .with_columns(pl.col('round.id').cast(pl.Int32))
            )     

In [39]:
#Get the each teams rolling 5 game strength based on margin
matchTeamStats = (allMatchesBothSidesStats
                    #get the previous 5 games margin average of opposition
                    #Start with descending round.id for rolling groupby statements
                    .sort(['team.A.name', 'round.id'], descending=[False, True])
                    #Try a rolling previous avg margin
                    .sort('team.A.name', 'round.id')
                    .rolling(index_column='round.id', period='5i', closed='left', group_by='team.A.name')
                    .agg(pl.col('team.A.score.totalScore').sum().alias('rolling.team.A.totalScore'),
                         pl.col('team.B.score.totalScore').sum().alias('rolling.team.B.totalScore'))
                    #get the avg margin
                    .with_columns(((pl.col('rolling.team.A.totalScore') - pl.col('rolling.team.B.totalScore'))/5).alias('rolling.margin.avg'))
                    #join back up to df
                    .join(allMatchesBothSidesStats, how='left', on=['team.A.name', 'round.id'])
                    #arrange columns as desired
                    .select('compSeason.year', 'round.roundNumber', 'team.A.name', 'strength.attack',
                            'strength.defense', 'rolling.margin.avg')
                    #Get the previous 5 matches attacking strength
                    .with_columns(pl.col('strength.attack').shift().over('team.A.name').alias('strength.attack.t-1'),
                                  pl.col('strength.attack').shift(2).over('team.A.name').alias('strength.attack.t-2'),
                                  pl.col('strength.attack').shift(3).over('team.A.name').alias('strength.attack.t-3'),
                                  pl.col('strength.attack').shift(4).over('team.A.name').alias('strength.attack.t-4'),
                                  pl.col('strength.attack').shift(5).over('team.A.name').alias('strength.attack.t-5'))
                    #Get the previous 5 matches defending strength
                    .with_columns(pl.col('strength.defense').shift().over('team.A.name').alias('strength.defense.t-1'),
                                  pl.col('strength.defense').shift(2).over('team.A.name').alias('strength.defense.t-2'),
                                  pl.col('strength.defense').shift(3).over('team.A.name').alias('strength.defense.t-3'),
                                  pl.col('strength.defense').shift(4).over('team.A.name').alias('strength.defense.t-4'),
                                  pl.col('strength.defense').shift(5).over('team.A.name').alias('strength.defense.t-5'))
                    #drop the original strength metrics
                    .drop('strength.attack', 'strength.defense')

                    
                )

In [40]:
#Attach the stats for the home team, i.e. own strength.attack, strength.defense and as away's opposition strength
seasonStep6 = (seasonStep5
     #let's do a bit of cleaning first remove some of the unneeded columns
     .drop('id', 'status', 'compSeason.id')
     #join for home team
     .join(matchTeamStats, how='left',
           left_on=['compSeason.year', 'round.roundNumber', 'home.team.name'],
           right_on=['compSeason.year', 'round.roundNumber', 'team.A.name'])
 
     #get past strengths of home team
     .rename({'strength.attack.t-1' : 'home.attack.t-1',
              'strength.attack.t-2' : 'home.attack.t-2',
              'strength.attack.t-3' : 'home.attack.t-3',
              'strength.attack.t-4' : 'home.attack.t-4',
              'strength.attack.t-5' : 'home.attack.t-5',
              'strength.defense.t-1' : 'home.defense.t-1',
              'strength.defense.t-2' : 'home.defense.t-2',
              'strength.defense.t-3' : 'home.defense.t-3',
              'strength.defense.t-4' : 'home.defense.t-4',
              'strength.defense.t-5' : 'home.defense.t-5',
              #For the away team the oppositions strength
              'rolling.margin.avg' : 'away.opposition.strength'})
 
      #join for away team
     .join(matchTeamStats, how='left',
           left_on=['compSeason.year', 'round.roundNumber', 'away.team.name'],
           right_on=['compSeason.year', 'round.roundNumber', 'team.A.name'])
 
     #get past strengths of away team
     .rename({'strength.attack.t-1' : 'away.attack.t-1',
              'strength.attack.t-2' : 'away.attack.t-2',
              'strength.attack.t-3' : 'away.attack.t-3',
              'strength.attack.t-4' : 'away.attack.t-4',
              'strength.attack.t-5' : 'away.attack.t-5',
              'strength.defense.t-1' : 'away.defense.t-1',
              'strength.defense.t-2' : 'away.defense.t-2',
              'strength.defense.t-3' : 'away.defense.t-3',
              'strength.defense.t-4' : 'away.defense.t-4',
              'strength.defense.t-5' : 'away.defense.t-5',
              #For the home team the oppositions strength
              'rolling.margin.avg' : 'home.opposition.strength'})
            
    )
 

## Player Rankings


In [41]:
%%R -o seasons
seasonList <- c(2016,2017,2018,2019,2020,2021, 2022, 2023, 2024, 2025)
first <- TRUE
for(s in seasonList) {
    
    df <- fetch_player_stats(season=s) %>% mutate('compSeason.year' = s) %>% select(compSeason.shortName, compSeason.year, round.roundNumber, teamId, team.name, player.playerId, player.givenName, player.surname, ratingPoints)

    if (first) {
        seasons <- df
        first <- FALSE
    }else{
        seasons <- seasons %>% 
                      bind_rows(df)
    }
}
    


ℹ Fetching match ids


✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [30.1s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [29.8s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [29.8s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [29.9s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 162 matches.
✔ Finding player stats for 162 matches. [20.9s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [27.4s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [26.3s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 216

In [42]:
seasons = replaceTeamNames(pl.from_pandas(seasons), 'team.name')

In [43]:
playerPoints = (seasons
                     #.drop('roundId')
                     .join(seasonsBase.select('compSeason.year', 'round.roundNumber', 'round.id'), how='left', on=['compSeason.year', 'round.roundNumber'])
                     .unique()
                     .with_columns(pl.col('round.id').cast(pl.Int32),
                                   pl.col('ratingPoints').fill_null(0))
                     .filter(pl.col('round.id').is_not_null())
               )

In [44]:
#Create the base of the player ratings
playerRatings = (playerPoints
     .sort('player.playerId', 'compSeason.year', 'round.roundNumber')
     .sort('player.playerId', 'round.id')
     #get sum of past 10 games
     .rolling(index_column='round.id', period='10i', group_by=['player.playerId', 'player.givenName', 'player.surname'])
     .agg(pl.col('ratingPoints').sum().alias('rolling.player.points'))
     .sort('player.playerId', 'round.id')
     #get the next round to join on
     .with_columns(pl.col('round.id').shift(-1).over('player.playerId').alias('round.nextRound'))
     #join back up to get the team
     .join(playerPoints.select('round.id', 'teamId', 'team.name', 'player.playerId'), on=['round.id', 'player.playerId'])
     .sort('player.playerId', 'round.id')
)

In [45]:
#get the sum of teams rating
sumedteamRating = (playerPoints
    .group_by('team.name', 'round.id')
    .agg(pl.col('ratingPoints').sum())
    .sort('team.name', 'round.id')
    .rolling(index_column='round.id', period='10i', group_by=['team.name'])
    .agg(pl.col('ratingPoints').sum().alias('rolling.ratingPoints'))
    .with_columns(pl.col('round.id').shift(-1).over('team.name').alias('nextRound'))
)

In [46]:
seasonStep7 = (seasonStep6
     .with_columns(pl.col('round.id').cast(pl.Int32))
     #join home teams ratings
     .join(sumedteamRating.drop('round.id'), how='left', left_on=['home.team.name', 'round.id'], right_on=['team.name', 'nextRound'])
     .rename({'rolling.ratingPoints' : 'home.team.playerPoints'})
     #join away tea,s ratings
     .join(sumedteamRating.drop('round.id'), how='left', left_on=['away.team.name', 'round.id'], right_on=['team.name', 'nextRound'])
     .rename({'rolling.ratingPoints' : 'away.team.playerPoints'})
)

## Keep and Order the final dataframe

In [47]:
dataFinal = (seasonStep7
             .select('round.id', 'compSeason.year', 'home.team.id', 'home.bye.flag', 'home.ladder.position', 'home.elo.score', 'home.venue.portion', 'home.day.last.match', 'home.attack.t-1', 'home.attack.t-2',
                   'home.attack.t-3', 'home.attack.t-4', 'home.attack.t-5', 'home.defense.t-1', 'home.defense.t-2', 'home.defense.t-3', 'home.defense.t-4', 'home.defense.t-5', 'home.opposition.strength',
                   'home.team.playerPoints', 'away.team.id', 'away.bye.flag', 'away.ladder.position', 'away.elo.score', 'away.venue.portion', 'away.day.last.match', 'away.attack.t-1', 'away.attack.t-2',
                   'away.attack.t-3', 'away.attack.t-4', 'away.attack.t-5', 'away.defense.t-1', 'away.defense.t-2', 'away.defense.t-3', 'away.defense.t-4', 'away.defense.t-5', 'away.opposition.strength',
                   'away.team.playerPoints', 'win.margin', 'winner')
             #remove 2015
             .filter(pl.col('compSeason.year') > 2016)
            )

## Model: Find the winner

In [48]:
#import packages for modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
import pickle

In [49]:
trainX = dataFinal.filter(pl.col('compSeason.year') != 2025).drop('win.margin', 'winner', 'compSeason.year', 'round.id', 'home.team.id', 'away.team.id')
trainY = dataFinal.filter(pl.col('compSeason.year') != 2025).select('round.id', 'home.team.id',
                                                                    'away.team.id', 'win.margin', 'winner')
trainWinner = trainY.select('winner').to_numpy().ravel()
trainMargin = trainY.select('win.margin').to_numpy().ravel()

In [50]:
testX = dataFinal.filter(pl.col('compSeason.year') == 2025).drop('win.margin', 'winner', 'compSeason.year',  'round.id', 'home.team.id', 'away.team.id')
testY = dataFinal.filter(pl.col('compSeason.year') == 2025).select('round.id', 'home.team.id',
                                                                    'away.team.id', 'win.margin', 'winner')
testWinner = testY.select('winner').to_numpy().ravel()
testMargin = testY.select('win.margin').to_numpy().ravel()

In [51]:
testY.describe()

statistic,round.id,home.team.id,away.team.id,win.margin,winner
str,f64,f64,f64,f64,f64
"""count""",80.0,80.0,80.0,80.0,80.0
"""null_count""",0.0,0.0,0.0,0.0,0.0
"""mean""",273.95,8.8625,8.2375,31.7625,0.5875
"""std""",2.699742,5.142879,5.304032,26.204765,0.49539
"""min""",269.0,0.0,0.0,0.0,0.0
"""25%""",272.0,5.0,4.0,10.0,0.0
"""50%""",274.0,9.0,8.0,20.0,1.0
"""75%""",276.0,13.0,13.0,52.0,1.0
"""max""",278.0,17.0,17.0,91.0,1.0


In [52]:
#Run some tests on pararmeters
n_estimators=[50,75,100,125,130, 140, 150]
max_depth=[None, 10, 13, 14, 15, 16, 17]
min_samples_split=[2,3, 4, 5,8]
max_features=['sqrt', 'log2', None]

In [53]:
#Test all combos and record the results
ne_list = []
mss_list = []
md_list = []
mf_list = []
result = []

for ne in n_estimators:
    for mss in min_samples_split:
            for mf in max_features:
                for md in max_depth:
                            #Create model
                            model = Pipeline([("Scaler", Normalizer()), ("Tree", RandomForestClassifier(n_estimators=ne, max_depth=md, min_samples_split=mss, max_features=mf, random_state=40))])
                            #Fit model
                            model.fit(trainX, trainWinner)
                            #Test checking
                            model.predict(testX)
                            testYMitPred = (testY
                                             .with_columns(pl.Series(model.predict(testX)).alias('Prediction'))
                                             .with_columns(pl.when(pl.col('winner') == pl.col('Prediction')).then(pl.lit(1)).otherwise(pl.lit(0)).alias('Check'))
                                            )
                            #get result
                            result.append(testYMitPred.select(pl.col('Check')).sum().item()/80)
                            #append parameters
                            ne_list.append(ne)
                            md_list.append(md)
                            mss_list.append(mss)
                            mf_list.append(mf)
                    
                    
#Display in df
paramTesting = pl.DataFrame({'n_estimators' : ne_list, 'max_depth' : md_list, 'Min_sample_split' : mss_list, 'Max_features' : mf_list, 
                             'Results': result})

In [54]:
paramTesting.sort('Results', descending=True).head()

n_estimators,max_depth,Min_sample_split,Max_features,Results
i64,i64,i64,str,f64
100,13,5,"""sqrt""",0.75
100,13,5,"""log2""",0.75
75,10,3,"""sqrt""",0.725
75,10,3,"""log2""",0.725
100,13,2,"""sqrt""",0.725


In [55]:
model = Pipeline([("Scaler", Normalizer()), ("Tree", RandomForestClassifier(n_estimators=100, min_samples_split=5, max_depth=13, max_features='sqrt', random_state=40))])
model.fit(dataFinal.drop('win.margin', 'winner', 'compSeason.year', 'round.id', 'home.team.id', 'away.team.id'), dataFinal.select('winner').to_numpy().ravel())


pickle.dump(model, open('Data/winner.sav', 'wb'))

In [56]:
probTrain = model.predict_proba(trainX)
trainX[['homeProb', 'awayProb']] = probTrain
trainX = (trainX
    .with_columns(pl.when(pl.col('homeProb') >= pl.col('awayProb')).then(pl.col('homeProb')).otherwise(pl.col('awayProb')).alias('WinProb'))
    #.drop('homeProb', 'awayProb')
)

In [57]:
probTest = model.predict_proba(testX)
testX[['homeProb', 'awayProb']] = probTest
testX = (testX
    .with_columns(pl.when(pl.col('homeProb') >= pl.col('awayProb')).then(pl.col('homeProb')).otherwise(pl.col('awayProb')).alias('WinProb'))
    #.drop('homeProb', 'awayProb')
)

### Save Bits and Pieces

In [58]:
#save everythingelse
indexTeams.write_excel('Data/indexTeams.xlsx')
distanceTravelled.write_excel('Data/distanceTravelled.xlsx')
with open("Data/eloScores", "wb") as fp:
    pickle.dump(eloScores[-1], fp)
(matchDate
     .with_columns(pl.col('team.round.id').max().over('Team').alias('MaxRound'))
     .filter(pl.col('team.round.id') == pl.col('MaxRound'))
     .select('Date', 'Team')).write_excel('Data/matchDay.xlsx')  
allMatchesBothSides.write_excel('Data/matchesBothSides.xlsx')

### Model: Margin

In [59]:
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
import pickle 

In [60]:
#Run some tests on pararmeters
splitter = ['best', 'random']
max_depth=[None, 10, 13, 14, 15, 16, 17]
min_samples_split=[2,3, 4, 5,8]
max_features=['sqrt', 'log2', None]

In [61]:
#Test all combos and record the results
s_list = []
mss_list = []
md_list = []
mf_list = []
result = []

for s in splitter:
    for mss in min_samples_split:
            for mf in max_features:
                for md in max_depth:
                            #Create model
                            model = Pipeline([('Scaler', StandardScaler()), ('Tree', DecisionTreeRegressor(splitter=s, max_depth=md, min_samples_split=mss, max_features=mf, random_state=20))])
                            #Fit model
                            model.fit(trainX.drop('WinProb'), trainMargin)
                            #Test checking
                            testYMitPred = (testY
                                             .with_columns(pl.Series(model.predict(testX.drop( 'WinProb'))).alias('Prediction'))
                                             .with_columns(((pl.col('win.margin') - pl.col('Prediction'))**2).alias('SqDiff'))
                                             .select(pl.col('SqDiff').sum().alias('SqDiffSum'), pl.col('SqDiff').len().alias('Len'))
                                             .with_columns(((1/pl.col('Len'))*pl.col('SqDiffSum')).alias('MSE'))
                                             .select('MSE')
                                             .item()
                                            )
                            #get result
                            result.append(testYMitPred)
                            #append parameters
                            s_list.append(s)
                            md_list.append(md)
                            mss_list.append(mss)
                            mf_list.append(mf)
                    
                    
#Display in df
paramTesting = pl.DataFrame({'splitter' : s_list, 'max_depth' : md_list, 'Min_sample_split' : mss_list, 'Max_features' : mf_list, 
                             'Results': result})

In [62]:
paramTesting.sort('Results', descending=False).head()

splitter,max_depth,Min_sample_split,Max_features,Results
str,i64,i64,str,f64
"""random""",10,2,"""log2""",693.421201
"""best""",13,4,"""sqrt""",712.408092
"""best""",14,5,"""sqrt""",716.060987
"""best""",13,5,"""sqrt""",716.129892
"""best""",10,3,"""sqrt""",775.105081


In [63]:
np.sqrt(670)

np.float64(25.88435821108957)

In [64]:
model = Pipeline([('Scaler', StandardScaler()), ('Tree', DecisionTreeRegressor(splitter='random', max_depth=10, min_samples_split=2, max_features="log2", random_state=20))])
model.fit(trainX.drop('WinProb'), trainMargin)
pickle.dump(model, open('Data/margin.sav', 'wb'))